<h1>Group 1 - Facebook<span class="tocSkip"></span>

# Introduction

One of the sources we need to scrap was the Facebook website (several pages).This notebook contains the code to retrieve the data and also the Robot to retrieve the data once a week. We scrapped posts on specifics pages. They contain for each publication, one comment, the publication date and number of comment, share and likes.

V0 : Soup scrap + click function

V2 : Comment recuperation + try of click on ("more comment") (only the first post is possible)

V4 : Recuperation of likes, comment and share number + description

V5 : Code formatting

# Environnement

## Libraries


In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common import action_chains
import re
import time
from datetime import datetime, timedelta

## Functions


This part contains all the functions we developped during the project

In [20]:
def dynamicSoup(url, link_driver, n_loop):
    """Documentation    
       Parameters:
            url : url page
            link_driver : driver link
            n_boucle : scroll time number
       out : 
            bs(driver.page_source, 'html.parser') : driver of the new page
            driver : driver
    """
    driver = webdriver.PhantomJS(link_driver)
    driver.set_window_size(1400,1000)
    driver.get(url)
    y = 0
        
    for _ in range(n_loop):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo("+str(y)+","+str(y+2000)+")")
        time.sleep(2)
        y += 2000
    return bs(driver.page_source, 'html.parser'), driver

In [21]:
def clickShow(driver):
    """Documentation    
       Parameters:
            driver : driver

    """
    driver.find_element_by_class_name("_3j0u").click()
    elements = driver.find_elements_by_class_name("_4ssp")
    action = action_chains.ActionChains(driver)
    for el in elements:
        time.sleep(2)
        try:
            action.move_to_element(el)
            time.sleep(4)
            action.click(el)
            action.perform()
        except:
            print("err")

In [22]:
def replace_all(text, dic):
    """Documentation    
       Parameters:
            text : character string
            dic : dictionary which contains the changes to be made
       out : 
            text : text with all changes made 
    """
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [19]:
def Simplification(chain):
    # transform M into 1000000,  and K into 1000
    """Documentation    
       Parameters:
            chain : character string in format "ex : 300K"    
       out : 
            int(float(expression) * coef) : integer  "ex 300 000"          
    """
    chain = chain.replace(',', '.')
    if '.' in chain:
        expression = (re.search("\d+\.\d+", chain)).group(0)
    else:
        expression = (re.search("\d+", chain)).group(0)
    coef = 1
    if 'K' in chain:
        coef = 1000
    if 'M' in chain:
        coef = 1000000
    return(int(float(expression) * coef))


In [18]:
def recovTextBetweenTags(texts: str, separator: str):
    """ Retrieving code between two tags
    
    Paramters:
        texts = Part of soup
        separator = Separator of soup
    Outers:
        description = Text wanted
    """ 
    text_clean = []
    lisI = []
    lisS = []

    for i in range(0, len(texts)):
        if str(texts[i]) == "<":
            lisI.append(i)
        if texts[i] == '>':
            lisS.append(i)

    len_lis = len(lisI)
    for h in range(0, len_lis):
        if h < (len_lis-1):
            text_clean.append(texts[lisS[h]:lisI[h+1]])

    if separator != 'non':
        description = str(text_clean).replace('>', '').replace(
            ',', '').replace('\'', '').replace('，', '')
        description = description.split(separator)
    else:
        description = text_clean

    return description

In [67]:
def CreateJs(listURL):
    """Documentation    
       Parameters:
            url1_page
            dic 
       out : 
            bs(driver.page_source, 'html.parser') : driver of the new page
            driver : driver
    """

    name_col = ['Data_Source',
                'Airline_Name',
                'Airline_Type',
                'Region_Operation',
                'Aircraft_Type',
                'Cabin_Class',
                'Type_Of_Lounge',
                'Type_Of_Traveller',
                'Date_Visit',
                'Date_Flown',
                'Airport',
                'Route',
                'Category',
                'Category_Detail',
                'Cabin_Staff_Service',
                'Lounge_Staff_Service',
                'Bar_And_Beverages',
                'Food_And_Beverages',
                'Ground_Service',
                'Catering',
                'Cleanliness',
                'Lounge_Comfort',
                'Aisle_Space',
                'Wifi_And_Connectivity',
                'Inflight_Entertainment',
                'Viewing_Tv_Screen',
                'Power_Supply',
                'Seat',
                'Seat_type',
                'Seat_Comfort',
                'Seat_Legroom',
                'Seat_Storage',
                'Seat_Width',
                'Seat_Recline',
                'Washrooms',
                'Value_For_Money',
                'Overall_Customer_Rating',
                'Overall_Service_Rating',
                'Overall_Airline_Rating',
                'Recommended',
                'Date_Review',
                'Review',
                'Departure_city',
                'Arrival_city',
                'Nb_bus_taken',
                'Nb_train_taken',
                'Nb_car_taken',
                'Nb_plane_taken',
                'Duration',
                'Price_min',
                'Price_max',
                'Title',
                'Author',
                'Description',
                'Date_publication',
                'View_Count',
                'Likes',
                'Dislikes',
                'Nb_subscribers',
                'Nb_comments',
                'Nb_sharing',
                'Hashtags',
                'Awards',
                'Registration',
                'Location',
                'Contributions_Pers',
                'Nb_pertinent_comments_Pers',
                'Queuing_Times',
                'Terminal_Seating',
                'Terminal_Signs',
                'Airport_Shopping',
                'Experience_At_Airport']

    dataAirline = pd.DataFrame(columns=name_col)

    lien_driver = "C:/Users/user/Desktop/phantomjs-2.1.1-windows/bin/phantomjs.exe"

    dic2 = {'.': '', 'avril': '04', 'janvier': '01',
            'mars': '03', 'mai': '05', 'juin': '06', 'février': '02', 'juillet': '07', 'décembre': '12',

            'août': '08', 'septembre': '09', 'aoÃ»t': '08', 'dÃ©c': '12', 'novembre': '11'}

    for ind, url in enumerate(listURL):
        ur = url + 'posts/?ref=page_internal'
        soup, driver = dynamicSoup(ur, lien_driver, 4)

        # get likes
        nb_like2 = []
        desc = []
        share2 = []
        date2 = []
        nb_com2 = []
        comment1 = []
        com2 = []
        source = []
        
        for span in soup.findAll('div', attrs={'class': '_5pcr userContentWrapper'}):

            nb_like = span.find('span', attrs={'class': '_81hb'})

            if nb_like == None:
                nb_like2.append(' ')
            else:
                nb_like2.append(
                    int(str((str(Simplification(str(nb_like).replace('\xa0', '').replace('>', '')))))))

            description = span.find(
                'div', attrs={'class': '_5pbx userContent _3576'})
            if description == None:
                desc.append(' ')
                source.append('Facebook')
            else:
                desc.append(translate(str(recovTextBetweenTags(str(description), 'non')).replace(
                    '\'>\',', '').replace('>', '')))
                source.append('Facebook')

            date1 = span.find(
                'span', attrs={'class': 'timestampContent', 'id': re.compile(r'js_')})
            if date1 == None:
                date2.append(' ')
            else:
                date2.append(recovTextBetweenTags(str(date1), 'non'))

            share1 = span.find('a', attrs={'class': '_3rwx _42ft'})
            if share1 == None:
                share2.append(' ')
            else:
                share2.append(recovTextBetweenTags(str(share1), 'non'))

            nb_com = soup.find('a', attrs={'class': '_3hg- _42ft'})
            if nb_com == None:
                nb_com2.append(' ')
            else:
                nb_com2.append(recovTextBetweenTags(str(nb_com), 'non'))

            com = span.find('span', attrs={'class': '_3l3x'})
            if com == None:
                com2.append(' ')
            else:
                com2.append(translate(str(recovTextBetweenTags(str(com), 'non')).replace(
                    '>,', '').replace('>', '').replace('\'\', ', '')))

        for i in range(0, len(date2)):
            if('h' in (date2[i])):
                date2[i] = str(date.today())
            else:
                date2[i] = replace_all(str(date2[i]), dic2)
                if (re.search("^[0-9]*\s[0-9]*,", str(date2[i]))) != None:
                    date2[i] = date2[i].split(",")[0]+' 2020'
                    date2[i] = str(datetime.strptime(re.search(
                        "[0-9]* [0-9]* [0-9]*", date2[i]).text.group(0), '%d %m %Y')).split(" ")[0]
        # get share
        for i in range(0, len(share2)):
            if share2[i] != ' ':
                share2[i] = int(Simplification(str(share2[i]).replace(
                    "partages", "").replace("\xa0", "").replace(" ", "").replace(">", "")))

        # get nb_com
        for i in range(0, len(nb_com2)):
            nb_com2[i] = Simplification(
                str(nb_com2[i]).replace(" commentaires", ""))

        # get comments
        comments = driver.find_elements_by_xpath(
            "//div [@data-testid='UFI2Comment/body']")
        for i in range(0, len(comments)):
            comment1.append(comments[i].text)
        
        df_template = pd.DataFrame({'Data_Source': source, 'Likes': nb_like2, 'Description': desc, 'Date_publication': date2,
                                    'Nb_sharing': share2, 'Nb_comments': nb_com2, 'Review': com2})
        dataAirline = pd.concat([dataAirline, df_template])

        print('page numéro ', ind, ' fini')

    return(dataAirline)

In [16]:
#Read the facebook pages to be scrapped in the pagesFb file, then scrape them one by one.

def Create_List_URL(chain):
    with open(chain, "r") as f :
    
        fb_pages = f.read()
        files = fb_pages.split("\n")
    return(files)

def addJSON(file: str, df, creat: bool):
    """
    Add of dataFrame in an existant JSON
    Parameters:
        file = path of JSON
        df = dataFrame of news datas
    Outers:
        creat = description of fly
    """
    
    if creat is False :
        with open(file) as train_file:
            dict_train = json.load(train_file)
        data = pd.read_json(dict_train, orient="records")
        df = pd.concat([data, df])
    
    js = df.to_json(orient='records').replace(
        "[\\\"[", '').replace("]\\\"]", '')
    
    with open(file, 'w', encoding='utf8') as outfile:
        json.dump(js, outfile, ensure_ascii=False, indent=4)

In [17]:
def translate (texte: str) : 
    """
    Paramters:
        texte = Text to be translated
    Outers:
        new = Text translated
    """
    try :
        new = str(TextBlob(texte).translate(to='en'))
        return new
    except :
        return texte

# Crawl

## Create JSON file

In [53]:
f = Create_List_URL("C:/Users/user/Desktop/pagesFb")
r = CreateJs(f[0:3],0)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:183: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



page numéro  0  fini
page numéro  1  fini
page numéro  2  fini


# Descriptive statistics on recovered data

In [34]:
with open('C:/Users/user/Desktop/ScrapFacebook.json') as train_file:
    dict_train = json.load(train_file)
data = pd.read_json(dict_train, orient="records")

In [43]:
data['Date_publication']

0           ['>17 01, 05:42']
1           ['>10 01, 09:51']
2            ['>1 01, 05:04']
3      ['>26 12 2019, 07:43']
4      ['>22 12 2019, 06:40']
                ...          
390    ['>30 12 2019, 05:08']
391    ['>28 12 2019, 02:44']
392    ['>26 12 2019, 05:43']
393    ['>25 12 2019, 04:35']
394    ['>24 12 2019, 04:45']
Name: Date_publication, Length: 395, dtype: object

In [26]:
data[['Nb_comments']].describe()

,Nb_comments
count,395.000000
mean,30.493671
std,50.895053
min,1.000000
25%,2.000000
50%,5.000000
75%,34.000000
max,184.000000
